<a href="https://colab.research.google.com/github/GiorgosMaragkopoulos/Quantum-data-encoding-using-QAE-PQAE/blob/main/Superdense_encoding_QAE_Iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler ,MinMaxScaler , Normalizer
from torch.nn.functional import normalize


# Load Iris dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert to PyTorch tensor before normalization
#X = torch.tensor(normalize(torch.tensor(X, dtype=torch.float32), p=2, dim=1), dtype=torch.float32)
normalizer = Normalizer(norm='l2')
X = normalizer.fit_transform(X)


# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)

# Define the autoencoder model
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(4, 2)  # 4 inputs to 3 nodes (bottleneck)
        )
        self.decoder = nn.Sequential(
            nn.Linear(2, 4),  # 3 nodes to 4 outputs
            nn.Linear(4, 4)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Instantiate the model, define loss function and optimizer
model = Autoencoder()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.001)

# Training the autoencoder
num_epochs = 10000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X)
    loss = criterion(outputs, X)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print progress
    if (epoch+1) % 500 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [500/10000], Loss: 0.2664
Epoch [1000/10000], Loss: 0.0423
Epoch [1500/10000], Loss: 0.0419
Epoch [2000/10000], Loss: 0.0419
Epoch [2500/10000], Loss: 0.0419
Epoch [3000/10000], Loss: 0.0419
Epoch [3500/10000], Loss: 0.0419
Epoch [4000/10000], Loss: 0.0419
Epoch [4500/10000], Loss: 0.0419
Epoch [5000/10000], Loss: 0.0419


KeyboardInterrupt: 

# Polynomial feature map

In [10]:
class Clifford_Autoencoder(nn.Module):
    def __init__(self):
        super(Clifford_Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(4, 2)  # 4 inputs to 2 nodes (bottleneck)
        )
        self.decoder = nn.Sequential(
            nn.Linear(3, 4)  # 3 nodes to 4 outputs
        )

        self.q0 = torch.tensor([[1], [0]], dtype=torch.cfloat)

    def forward(self, x):
        x = self.encoder(x)

        # Polynomial feature map
        x1_square = x[:, 0:1] ** 2
        x2_square = x[:, 1:2] ** 2
        x1_x2 = x[:, 0:1] * x[:, 1:2]

        # Combine the polynomial features
        poly_features = torch.cat((x1_square, x2_square, x1_x2 ), dim=-1)

        x = self.decoder(poly_features)

        return x

  # Instantiate the model, define loss function and optimizer
model = Clifford_Autoencoder()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.001)

# Training the autoencoder
num_epochs = 10000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X)
    loss = criterion(outputs, X)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print progress
    if (epoch+1) % 500 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [500/10000], Loss: 0.2037
Epoch [1000/10000], Loss: 0.1835
Epoch [1500/10000], Loss: 0.1435
Epoch [2000/10000], Loss: 0.1095
Epoch [2500/10000], Loss: 0.0847
Epoch [3000/10000], Loss: 0.0679
Epoch [3500/10000], Loss: 0.0578
Epoch [4000/10000], Loss: 0.0519
Epoch [4500/10000], Loss: 0.0484
Epoch [5000/10000], Loss: 0.0464
Epoch [5500/10000], Loss: 0.0452
Epoch [6000/10000], Loss: 0.0444
Epoch [6500/10000], Loss: 0.0439
Epoch [7000/10000], Loss: 0.0434
Epoch [7500/10000], Loss: 0.0431
Epoch [8000/10000], Loss: 0.0428
Epoch [8500/10000], Loss: 0.0426
Epoch [9000/10000], Loss: 0.0424
Epoch [9500/10000], Loss: 0.0422
Epoch [10000/10000], Loss: 0.0421


In [11]:



class Clifford_Autoencoder(nn.Module):
    def __init__(self):
        super(Clifford_Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(4, 2)  # 4 inputs to 2 nodes (bottleneck)
        )
        self.decoder = nn.Sequential(
            nn.Linear(5, 4)  # 3 nodes to 4 outputs
        )

        self.q0 = torch.tensor([[1], [0]], dtype=torch.cfloat)



    def forward(self, x):
        x = self.encoder(x)

        # Precompute trigonometric functions for all samples
        cos_x0 = torch.cos(x[:, 0] / 2).unsqueeze(-1)
        sin_x0 = torch.sin(x[:, 0] / 2).unsqueeze(-1)
        cos_x1 = torch.cos(x[:, 1])
        sin_x1 = torch.sin(x[:, 1])

        # Create the encoded tensor for all samples
        encoded = torch.zeros(x.size(0), 2, 2, dtype=torch.cfloat, device=x.device)
        encoded[:, 0, 0] = cos_x0.squeeze()
        encoded[:, 1, 0] = (cos_x1 + 1j * sin_x1) * sin_x0.squeeze()

        # Apply matrix multiplication for all samples
        qubit_1 = torch.matmul(encoded, self.q0).squeeze(-1)

        # Extract real and imaginary parts for logits
        real_part_1 = qubit_1.real
        imaginary_part_1 = qubit_1.imag[:, 1:2]
        x1= x[:,0:1]
        x2 = x[:,1:2]
        # Concatenate real and imaginary parts
        logits = torch.cat((real_part_1, imaginary_part_1,x1,x2), dim=-1)

        x = self.decoder(logits)

        return x

# Instantiate the model, define loss function and optimizer
model = Clifford_Autoencoder()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.001)

# Training the autoencoder
num_epochs = 10000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X)
    loss = criterion(outputs, X)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print progress
    if (epoch+1) % 500 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [500/10000], Loss: 0.1115
Epoch [1000/10000], Loss: 0.0473
Epoch [1500/10000], Loss: 0.0389
Epoch [2000/10000], Loss: 0.0363
Epoch [2500/10000], Loss: 0.0351
Epoch [3000/10000], Loss: 0.0347
Epoch [3500/10000], Loss: 0.0345
Epoch [4000/10000], Loss: 0.0345
Epoch [4500/10000], Loss: 0.0344
Epoch [5000/10000], Loss: 0.0344
Epoch [5500/10000], Loss: 0.0344
Epoch [6000/10000], Loss: 0.0343
Epoch [6500/10000], Loss: 0.0343
Epoch [7000/10000], Loss: 0.0343
Epoch [7500/10000], Loss: 0.0343
Epoch [8000/10000], Loss: 0.0343
Epoch [8500/10000], Loss: 0.0343
Epoch [9000/10000], Loss: 0.0343
Epoch [9500/10000], Loss: 0.0343
Epoch [10000/10000], Loss: 0.0343


In [ ]:
9

9